In [ ]:
import os, sys
import h5py
import numpy as np
import xarray 
import rioxarray
import holoviews as hv
from holoviews import opts
import geoviews as gv
import geopandas as gpd
import datashader as ds
import cartopy.crs as ccrs
import hvplot
import hvplot.pandas
from holoviews.operation.datashader import regrid, shade
from bokeh.tile_providers import STAMEN_TONER
import rasterio
from osgeo import gdal
from pathlib import Path
import panel
import hvplot.xarray
from ipywidgets import interact, Dropdown, FloatSlider, IntSlider, SelectMultiple, Text
import matplotlib.pyplot as plt
import xmltodict
import zipfile
import tarfile
import pandas
import cv2
from PIL import Image
from skimage import exposure, img_as_float

# Import functions from common hsi functions file
sys.path.insert(0, os.path.abspath('../../'))
import speclib

hv.extension('bokeh', width=1000)

# EnMap 

## Download from FTP

In [ ]:
usernameText = Text(value='username')
passwordText = Text(value='password')
display(usernameText)
display(passwordText)

In [ ]:
username = usernameText.value
password = passwordText.value

In [ ]:
print(username, password)

In [ ]:
!wget --user=$username --password=$password ftps://simonaoliver-cat1-A00001-P00348@download.dsda.dlr.de//dims_op_oc_oc-en_701103521_1.tar.gz

## Unzip

In [ ]:
targzfiles = []
targzfiles += [each for each in os.listdir() if each.endswith('.tar.gz')]

In [ ]:
targzfiles

In [ ]:
# Extract the Zip archive from the tar.gz - extract the required files from the Zip archive
# TODO - fairly sure the logic here is not correct and unzips are happening for existing files
filenames = []
for filename in targzfiles:
    enmaptarfile = tarfile.open(filename, 'r')
    for i in enmaptarfile.getnames():
        if 'ZIP' in i:
            print(i)
            enmaptarfilezip = i
            enmaptarfile.extract(enmaptarfilezip)
            enmapzipfile = zipfile.ZipFile(enmaptarfilezip, 'r')
            if not Path(enmapzipfile.filename).is_file():
                print(enmapzipfile.filename,' does not exist, untarring...')
                enmaptarfile.extract(enmapzipfile.filename)
            
            for index, value in enumerate(enmapzipfile.filelist):
                if 'SPECTRAL_IMAGE' in str(value.filename):
                    enmapspectralzipfile = value.filename
                if 'METADATA.XML' in str(value.filename):
                    enmapmetadatazipfile = str(value.filename)

            if not Path(enmapspectralzipfile).is_file():
                print(enmapspectralzipfile, ' does not exist, unzipping...')
                filename = enmapzipfile.extract(enmapspectralzipfile) 
            else: 
                filename = enmapspectralzipfile
            
            if not Path(enmapmetadatazipfile).is_file():
                print(enmapmetadatazipfile, ' does not exist, unzipping...')
                filenamexml = enmapzipfile.extract(enmapmetadatazipfile)
            else:
                filenamexml = enmapmetadatazipfile
            
            file = filename.split('/')[-1]
            filenames.append({'file':file,'filename':filename,'xml': filenamexml})

In [ ]:
# prepare filenames for dropdown list
filenamespd = pandas.DataFrame(filenames)

## Select file from dropdown list

In [ ]:
# Display dropdown list for file selection
# TODO enable multiple filename selects
#filenamesText =  SelectMultiple(options = list(filenamespd.file), layout={'width': 'max-content'})
filenamesText =  Dropdown(options = list(filenamespd.file), layout={'width': 'max-content'})
display(filenamesText)

In [ ]:
filename = filenamespd[filenamespd['file']==filenamesText.value].reset_index().filename[0]
filenamexml = (filenamespd[filenamespd['file']==filenamesText.value]).reset_index().xml[0]
filename

In [ ]:
#!gdalinfo $filename

In [ ]:
# Prepare dict from metadata and extract band list
fileptr = open(filenamexml,"r")
xml_content= fileptr.read()
enmapdict = xmltodict.parse(xml_content)
vnirlist = enmapdict['level_X']['specific']['vnirProductQuality']['expectedChannelsList']
swirlist = enmapdict['level_X']['specific']['swirProductQuality']['expectedChannelsList']
filenames.append({'file':file,'filename':filename,'xml': filenamexml, 'vnirlist': vnirlist, 'swirlist':swirlist})

In [ ]:
#enmapdict

## Create useful band names and apply to GeoTIFF

In [ ]:
bands = []
wavelengths = []
fwhms = []
for item in enmapdict['level_X']['specific']['bandCharacterisation']['bandID']:

    bandnumber = (int(item['@number']))
    if str(bandnumber) in vnirlist:
        label = 'VNIR'
    else:
        label = 'SWIR'
    centrewavelength = str(int(item['wavelengthCenterOfBand'].split('.')[0]))
    #bands.append((bandnumber, label+str(bandnumber)+' '+centrewavelength, '{"wavelength": "'+str(bandnumber)+'" }, "WAVELENGTH"'))
    bands.append((bandnumber, label+str(bandnumber)+' '+centrewavelength, str(item)+"WAVELENGTH"))
    wavelengths.append(float(item['wavelengthCenterOfBand']))
    fwhms.append(float(item['FWHMOfBand']))

In [ ]:
#enmapdict['level_X']['product']

In [ ]:
# Update the GeoTIFF band names to enable easier correspondence with other bands - use cwl for bands from metadata
speclib.set_band_descriptions(filename, bands, -32768)

In [ ]:
#gdal.Info(filename).split('\n')

In [ ]:
geofilename = filename.replace('.TIF', '_geo.TIF')
mercfilename = filename.replace('.TIF', '_3857.TIF')

In [ ]:
geofilename, mercfilename

## Warp GeoTIFF to lat/lon

In [ ]:
if not Path(geofilename).is_file():
    gdal.Warp(geofilename, filename, options="-t_srs EPSG:4326 -overwrite -tr 0.0003 0.0003 -wo WRITE_FLUSH=YES")
if not Path(mercfilename).is_file():
    gdal.Warp(mercfilename, filename, options="-t_srs EPSG:3857 -overwrite -wo WRITE_FLUSH=YES")

In [ ]:
enmapxarray = xarray.open_dataset(filename)

In [ ]:
enmapxarraygeo = xarray.open_dataset(geofilename)

In [ ]:
enmapxarraygeo

In [ ]:
enmapxarraymerc = xarray.open_dataset(mercfilename)

In [ ]:
enmapxarraygeo = updatexarray(enmapxarraygeo, 'bands', 'longitude', 'latitude', wavelengths, fwhms)
enmapxarraymerc = updatexarray(enmapxarraymerc, 'bands', 'x', 'y', wavelengths, fwhms)

In [ ]:
def updatexarray(ds, banddim, xdim, ydim, wavelengthlist, fwhmlist): 
    if xdim !='x' and ydim != 'y':
        ds = ds.rename_dims({'band':banddim, 'x': xdim,'y':ydim})
        ds = ds.rename({'x':xdim, 'y':ydim})
    ds = ds.assign_coords({'wavelengths':wavelengthlist, 'fwhm': fwhmlist})
    ds = ds.rename({'band':banddim,'band_data':'reflectance'})
    ds = ds.drop_indexes(['wavelengths', 'fwhm'])
    ds = ds.drop_indexes('bands')
    return(ds)

In [ ]:
#(enmapxarray.band_data[[50,145,195]]/40).hvplot.rgb( geo=True, x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)
"""
 'quicklook': {'vnir': {'channels': '3',
   'qlChannels': {'red': '635.112', 'green': '550.687', 'blue': '463.73'},
   'name': 'ENMAP01-____L2A-DT0000050503_20231118T011859Z_001_V010400_20231120T120732Z-QL_VNIR.TIF',
   'size': {'@unit': 'Kbyte', '#text': '2127'},
   'version': '01.04.00',
   'format': 'binary',
   'dimension': {'columns': '1286', 'rows': '1165'},
   'dimensionGeographic': {'longitude': {'@unit': 'DEG', '#text': '0.3129176'},
    'latitude': {'@unit': 'DEG', '#text': '0.3129176'}}},
  'swir': {'channels': '3',
   'qlChannels': {'red': '2199.45', 'green': '1653', 'blue': '1047.84'},
"""
# Get red, green and blue bands and plot example
b = np.nanargmin(abs(enmapxarraygeo['wavelengths'].data-463.73))
g = np.nanargmin(abs(enmapxarraygeo['wavelengths'].data-550.687))
r = np.nanargmin(abs(enmapxarraygeo['wavelengths'].data-635.112))

speclib.rgbhsi(enmapxarray.band_data, r, g, b, 0.1, 0.99, 'band').hvplot.rgb( geo=True, x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)

In [ ]:
#SWIR {'red': '2199.45', 'green': '1653', 'blue': '1047.84'},

b = np.nanargmin(abs(enmapxarraygeo['wavelengths'].data-1047.84))
g = np.nanargmin(abs(enmapxarraygeo['wavelengths'].data-1653))
r = np.nanargmin(abs(enmapxarraygeo['wavelengths'].data-2199.45))

speclib.rgbhsi(enmapxarraygeo.reflectance, r, g, b, 0.1, 0.99,'bands').hvplot.rgb( geo=True, x='longitude', y='latitude', bands='bands', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200, tiles='ESRI', alpha=0.6)

## Make an interactive RGB plot

In [ ]:
redW = Dropdown(options = list(enmapxarray.band.values))
greenW = Dropdown(options = list(enmapxarray.band.values))
blueW = Dropdown(options = list(enmapxarray.band.values))
startW = Dropdown(options = ['Pause', 'Go'])

@interact(red = redW, green = greenW, blue = blueW, start = startW)

def rgb_combo(red, green, blue, start):
    redW.options = list(enmapxarray.band.values)
    greenW.options = list(enmapxarray.band.values)
    blueW.options = list(enmapxarray.band.values)
    
    if start == 'Go' and red != green and green !=blue and red != blue:
        #rgbhsi(enmapxarraygeo.band_data, red, green, blue, 0.1, 0.99).hvplot.rgb( geo=True, x='x', y='y', bands='band',  title="RGB Plot with HVPlot", width=1200, tiles='ESRI', alpha=0.5)
        #enmapxarraygeo.band_data[[red, green, blue]].hvplot.rgb( geo=True, x='x', y='y', bands='band',  title="RGB Plot with HVPlot", width=1200, tiles='ESRI', alpha=0.5)
        #speclib.rgbhsi(enmapxarraygeo.band_data, red, green, blue, 0.1, 0.99)
        speclib.rgbhsi(enmapxarraygeo.reflectance, red, green, blue, 0.1, 0.99,'bands').hvplot.rgb( geo=True, x='longitude', y='latitude', bands='bands', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200, tiles='ESRI', alpha=0.6)
    return(red,green,blue,start)

# Configure for Open Data Cube

In [ ]:
# TODO
# https://eodatasets.readthedocs.io/en/latest/
# Write metadata
from eodatasets3 import DatasetPrepare

dlr_enmapl2d = '/mnt/c/Users/Simon/PycharmProjects/datacube.spectral/imagery/enmap/ENMAP01-____L2A-DT0000050503_20231118T011859Z_001_V010400_20231120T120732Z/'
metadata_path = dlr_enmapl2d+'/odc-metadata.yaml'

with DatasetPrepare(
    metadata_path=metadata_path,
) as p:
    p.product_family = "l2d"
    p.datetime = datetime(2019, 7, 4, 13, 7, 5)
    p.processed_now()

    # Note the measurement in the metadata. (instead of ``write``)
    p.note_measurement('red',
      enmapl2a / 'LC08_L1TP_090084_20160121_20170405_01_T1_B4.TIF'
    )

    # Or give the path relative to the dataset location
    # (eg. This will work unchanged on non-filesystem locations, such as ``s3://`` or tar files)
    p.note_measurement('blue',
       'LC08_L1TP_090084_20160121_20170405_01_T1_B2.TIF',
       relative_to_dataset_location=True
    )

    # Add links to other files included in the package ("accessories"), such as
    # alternative metadata files.
    [mtl_path] = usgs_level1.glob('*_MTL.txt')
    p.note_accessory_file('metadata:mtl', mtl_path)

    # Add whatever else you want.
    ...

    # Validate and write our metadata document!
    p.done()

# We created a metadata file!
assert metadata_path.exists()

# Get coincident Landsat and Sentinel-2 ARD

In [ ]:
# https://docs.dea.ga.gov.au/notebooks/How_to_guides/Downloading_data_with_STAC/
import urllib.request, json
import geopandas as gpd
import xarray as xr
import rioxarray
from pprint import pprint
import boto3
# ODC mamba/conda build fail - using wget on http instead
#import odc.aws
#import odc.geo.xr
from datacube.testutils.io import rio_slurp_xarray

In [ ]:
starttime = enmapdict['level_X']['base']['temporalCoverage']['startTime']
ul_lat = float(enmapdict['level_X']['base']['spatialCoverage']['boundingPolygon']['point'][0]['latitude']['#text'])
ul_lon = float(enmapdict['level_X']['base']['spatialCoverage']['boundingPolygon']['point'][0]['longitude']['#text'])
ll_lat = float(enmapdict['level_X']['base']['spatialCoverage']['boundingPolygon']['point'][1]['latitude']['#text'])
ll_lon = float(enmapdict['level_X']['base']['spatialCoverage']['boundingPolygon']['point'][1]['longitude']['#text'])
lr_lat = float(enmapdict['level_X']['base']['spatialCoverage']['boundingPolygon']['point'][2]['latitude']['#text'])
lr_lon = float(enmapdict['level_X']['base']['spatialCoverage']['boundingPolygon']['point'][2]['longitude']['#text'])
ur_lat = float(enmapdict['level_X']['base']['spatialCoverage']['boundingPolygon']['point'][3]['latitude']['#text'])
ur_lon = float(enmapdict['level_X']['base']['spatialCoverage']['boundingPolygon']['point'][3]['longitude']['#text'])

In [ ]:
product = 'ga_ls8c_ard_3'
products = ['ga_ls8c_ard_3', 'ga_ls9c_ard_3', 'ga_s2bm_ard_3', 'ga_s2bm_ard_3']
start_time = starttime.split('T')[0]
end_time = start_time

bbox = [min([ul_lon,ll_lon,lr_lon, ur_lon]), max([ul_lat,ll_lat,lr_lat, ur_lat]), max([ul_lon,ll_lon,lr_lon, ur_lon]),  min([ul_lat,ll_lat,lr_lat, ur_lat])]

In [ ]:
from datetime import datetime, timedelta
start = datetime.strptime(start_time, "%Y-%m-%d")-timedelta(days=3)
end = datetime.strptime(start_time, "%Y-%m-%d")+timedelta(days=3)

In [ ]:
start_time = start.strftime("%Y-%m-%d")
end_time = end.strftime("%Y-%m-%d")
start_time, end_time

In [ ]:
root_url = 'https://explorer.dea.ga.gov.au/stac'
stac_url = f'{root_url}/search?collection={product}&time={start_time}/{end_time}&bbox={str(bbox).replace(" ", "")}&limit=6'
print(stac_url)

In [ ]:
with urllib.request.urlopen(stac_url) as url:
    data = json.loads(url.read().decode())
pprint(data, depth=1)

In [ ]:
#pprint(data['features'], depth=2)

In [ ]:
# Convert features to a GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(data['features'])

# Plot the footprints of each dataset
gdf.plot(alpha=0.8, edgecolor='black')

In [ ]:
gdf

In [ ]:
# Colour features by cloud cover
gdf.plot(column='eo:cloud_cover',
         cmap='viridis',
         alpha=0.8,
         edgecolor='black',
         legend=True)

In [ ]:
stac_item = data['features'][0]
pprint(stac_item['assets'], depth=1)

In [ ]:
pprint(stac_item['assets']['nbart_blue'])

In [ ]:
filelist = []
for stac_item in data['features']:
     href = stac_item['assets']['thumbnail:nbart']['href']
     href = href.replace('s3://dea-public-data', 'https://data.dea.ga.gov.au')
     filelist.append(href.split('/')[-1])

In [ ]:
for file in filelist:
    print(file)
    !wget $href --no-clobber --no-parent

In [ ]:
image = Image.open(filename)
image.show()

# Get AOI extent

In [ ]:
import glob
import os
shplist = []
# glob.glob() return a list of file name with specified pathname
for file in glob.glob(r'../aoi/' + '*.shp', recursive=True):
  # print the path name of selected files
    print(os.path.join(r'../aoi/', file))
    target = file.split('/')[-1].replace(".shp", '')
    tgpd = gpd.GeoDataFrame.from_file(file)
    tgpd['Name']=target
    shplist.append(tgpd)
aoi = pandas.concat(shplist)


In [ ]:
aoi.plot()

In [ ]:
# Select only the AOI that intersects our target hyperspectral image
taoi = aoi.cx[min([ul_lon,ll_lon,lr_lon, ur_lon]): max([ul_lon,ll_lon,lr_lon, ur_lon]), min([ul_lat,ll_lat,lr_lat, ur_lat]):max([ul_lat,ll_lat,lr_lat, ur_lat])]
taoi = taoi.to_crs('EPSG:3857')
gdf = gdf.set_crs('EPSG:4326')
tgdf = gdf.to_crs('EPSG:3857')

In [ ]:
taoi

In [ ]:
(speclib.rgbhsi(enmapxarraygeo.band_data, 40, 120, 220, 0.15, 0.99).hvplot.rgb( geo=True, x='x', y='y', bands='band',  title="RGB Plot with HVPlot", width=1200, height=1000, tiles='ESRI')) *\
tgdf.hvplot.polygons(geo=False, alpha=0.2, hover_cols=['title']) *\
taoi.hvplot.polygons(geo=False, alpha=0.5, hover_cols=['Name'])